In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# read csv into df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
from glob import glob
from datetime import datetime
import os


df = pd.DataFrame(columns=['datum', "location_id", "pretocnost"])

location_dict = {
    "kum": 0,
    "lovrenška_jezera": 1,
    "osp": 2,
    "storžič": 3,
    "triglavski_narodni_park": 4,
    "vršič": 5,
}

for path in glob(DATA_GLOB_PLANINSKE_MAKRO_ALL):
    print(path)

    # get file name without extension and path
    file_name = os.path.basename(path).split('.')[0].lower().replace(" ", "_")
    if file_name == "zdruzeno":
        continue
    
    # file_name = path.split('/')[-1].split('.')[0].replace(" ", "_").lower()
    assert file_name in location_dict.keys(), f"file_name = {file_name}"
    print(file_name)
    location_id = location_dict[file_name]

    df_planine = pd.read_csv(path, sep=',', encoding='UTF-8')
    assert df_planine.columns[0] == 'datum', f"columns[0] = {df_planine.columns[0]}"
    assert df_planine.columns[1] == 'vhodi', f"columns[1] = {df_planine.columns[1]}"
    assert df_planine.columns[2] == 'izhodi', f"columns[2] = {df_planine.columns[2]}"
    
    # sum vhodi and izhodi
    df_planine['pretocnost'] = df_planine['vhodi'] + df_planine['izhodi']
    # drop "vhodi" and "izhodi" columns
    df_planine = df_planine.drop(columns=['vhodi', 'izhodi'])

    # add "location_id" column
    df_planine['location_id'] = location_id

    df = pd.concat([df, df_planine], ignore_index=True)

    # sum vhodi and izhodi based on "datum_dan"
    # df_planine = df_planine.groupby('datum').sum()
    print(df_planine.head())

    
    # break

df = df.sort_values(by=['datum', 'location_id'])
print(df.head())

df.to_csv(OUT_PLANINSTVO, index=False)



../data/planinstvo/2024\Kum.csv
kum
        datum  pretocnost  location_id
0  2022-06-20           3            0
1  2022-06-21          24            0
2  2022-06-22           3            0
3  2022-06-23           5            0
4  2022-06-24           8            0
../data/planinstvo/2024\Lovrenška jezera.csv
lovrenška_jezera
        datum  pretocnost  location_id
0  2022-06-27         318            1
1  2022-06-28          66            1
2  2022-06-29         214            1
3  2022-06-30         288            1
4  2022-07-01         379            1
../data/planinstvo/2024\Osp.csv
osp
        datum  pretocnost  location_id
0  2022-06-15          46            2
1  2022-06-16          60            2
2  2022-06-17          79            2
3  2022-06-18          62            2
4  2022-06-19          40            2
../data/planinstvo/2024\Storžič.csv
storžič
        datum  pretocnost  location_id
0  2022-07-19          72            3
1  2022-07-20         144            3
2  

## Add day of week to the data

In [41]:
import datetime

# convert datum to day of the week number with Monday=0, Sunday=6
df['datum'] = pd.to_datetime(df['datum'])
df['day_of_week'] = df['datum'].dt.dayofweek

# 

print(df.head())

          datum location_id pretocnost  day_of_week
2251 2022-06-01           4       1288            2
2252 2022-06-02           4        743            3
2253 2022-06-03           4       1128            4
2254 2022-06-04           4       2488            5
2255 2022-06-05           4       2466            6


## Add is holiday to the data

In [42]:
df_prazniki = pd.read_csv(OUT_PRAZNIKI)

# based on datum column from df add "praznik" column from df_prazniki. join based on "datum" column that is only the date without time
df_prazniki['datum'] = pd.to_datetime(df_prazniki['datum'])
df['datum'] = pd.to_datetime(df['datum'])
df_prazniki['datum'] = df_prazniki['datum'].dt.date
df['datum'] = df['datum'].dt.date
df_new = pd.merge(df, df_prazniki, on='datum', how='left') 

# fill NaN values in "dela_prost_dan" column with False
df_new['dela_prost_dan'] = df_new['dela_prost_dan'].fillna(False)
df_new


C:\Users\am8130\AppData\Local\Temp\ipykernel_13932\3376327649.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new['dela_prost_dan'] = df_new['dela_prost_dan'].fillna(False)


,datum,location_id,pretocnost,day_of_week,dela_prost_dan
0,2022-06-01,4,1288,2,False
1,2022-06-02,4,743,3,False
2,2022-06-03,4,1128,4,False
3,2022-06-04,4,2488,5,False
4,2022-06-05,4,2466,6,True
...,...,...,...,...,...
2857,2024-03-29,5,0,4,False
2858,2024-03-30,0,0,5,False
2859,2024-03-30,1,30,5,False
2860,2024-03-30,2,0,5,False


In [43]:
# sort by "location_id" and "datum"
df_new = df_new.sort_values(by=['location_id', 'datum'])
df_new

,datum,location_id,pretocnost,day_of_week,dela_prost_dan
24,2022-06-20,0,3,0,False
27,2022-06-21,0,24,1,False
30,2022-06-22,0,3,2,False
33,2022-06-23,0,5,3,False
36,2022-06-24,0,8,4,False
...,...,...,...,...,...
2845,2024-03-26,5,0,1,False
2849,2024-03-27,5,0,2,False
2853,2024-03-28,5,0,3,False
2857,2024-03-29,5,0,4,False


In [44]:
# add month column
df_new['month'] = pd.to_datetime(df_new['datum']).dt.month
df_new

,datum,location_id,pretocnost,day_of_week,dela_prost_dan,month
24,2022-06-20,0,3,0,False,6
27,2022-06-21,0,24,1,False,6
30,2022-06-22,0,3,2,False,6
33,2022-06-23,0,5,3,False,6
36,2022-06-24,0,8,4,False,6
...,...,...,...,...,...,...
2845,2024-03-26,5,0,1,False,3
2849,2024-03-27,5,0,2,False,3
2853,2024-03-28,5,0,3,False,3
2857,2024-03-29,5,0,4,False,3


In [45]:
# todo: add moving average by date for each location separately
df_new['pretocnost_7d'] = df_new.groupby('location_id')['pretocnost'].transform(lambda x: x.rolling(7, 1).mean())


In [46]:
# save to csv
df_new.to_csv(OUT_PLANINSTVO_END, index=False)


In [47]:
# split data to train (2022 and 2023) and test (2024)
df_train = df_new[df_new['datum'] < datetime.date(2024, 1, 1)]
df_test = df_new[df_new['datum'] >= datetime.date(2024, 1, 1)]

df_test

#split train on train and validation 80/20 split
# train_df, val_df = df_train(df_train, test_size=0.2, random_state=42)


,datum,location_id,pretocnost,day_of_week,dela_prost_dan,month,pretocnost_7d
2545,2024-01-01,0,28,0,True,1,57.857143
2549,2024-01-02,0,375,1,True,1,89.571429
2553,2024-01-03,0,1,2,False,1,82.142857
2557,2024-01-04,0,22,3,False,1,80.142857
2561,2024-01-05,0,7,4,False,1,70.285714
...,...,...,...,...,...,...,...
2845,2024-03-26,5,0,1,False,3,0.000000
2849,2024-03-27,5,0,2,False,3,0.000000
2853,2024-03-28,5,0,3,False,3,0.000000
2857,2024-03-29,5,0,4,False,3,0.000000


In [49]:
# remove "datum" column
df_train = df_train.drop(columns=['datum'])


KeyError: "['datum'] not found in axis"

In [51]:
# start training a model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df_train.drop(columns=['pretocnost'])
y = df_train['pretocnost']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=5000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"mse = {mse}")


# save model
import joblib
joblib.dump(model, OUT_MODEL)


mse = 40272187.83867017


['../models/RandomForestClassifier']